# Evaluation
- For retriever
- For generator

## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../../.."
sys.path.append(os.path.abspath(module_path))

## 1. Bedrock Client 생성

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [4]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

### LLM 로딩 (Claude-v2)

In [5]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [6]:
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    streaming=False,
    callbacks=[StreamingStdOutCallbackHandler()]
)

llm_text_light = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-Instant-V1"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    streaming=False,
    callbacks=[StreamingStdOutCallbackHandler()]
)

### Embedding 모델 선택

In [7]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

In [8]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):

    if is_bedrock_embeddings:
        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
                #model_name="Cohere-Embeddings-En"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")

    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [9]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)   

Bedrock Embeddings Model Loaded


## 3. Depoly ReRanker model (if needed)

In [10]:
import json
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [11]:
depoly = False

In [12]:
if depoly:

    try:
        role = sagemaker.get_execution_role()
    except ValueError:
        iam = boto3.client('iam')
        role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

    # Hub Model configuration. https://huggingface.co/models
    hub = {
        'HF_MODEL_ID':'BAAI/bge-reranker-large',
        'HF_TASK':'text-classification'
    }

    # create Hugging Face Model Class
    huggingface_model = HuggingFaceModel(
        transformers_version='4.26.0',
        pytorch_version='1.13.1',
        py_version='py39',
        env=hub,
        role=role, 
    )

    # deploy model to SageMaker Inference
    predictor = huggingface_model.deploy(
        initial_instance_count=1, # number of instances
        instance_type='ml.g5.xlarge' # instance type
    )

    print(f'Accept: {predictor.accept}')
    print(f'ContentType: {predictor.content_type}')
    print(f'Endpoint: {predictor.endpoint}')

### Reranker

In [13]:
#endpoint_name = "huggingface-pytorch-inference-2023-11-15-04-37-45-120" # ml.m5.2xlarge
endpoint_name = "huggingface-pytorch-inference-2023-11-15-07-53-21-605" # ml.g5.xlarge

In [14]:
runtime_client = boto3.Session().client('sagemaker-runtime')
print (f'runtime_client: {runtime_client}')

runtime_client: <botocore.client.SageMakerRuntime object at 0x7f97e92959f0>


## 4. LangChainmOpenSearch VectorStore 정의
### 선수 조건
- 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [15]:
from utils.proc_docs import get_parameter

In [16]:
ssm = boto3.client("ssm", "us-east-1")

opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_userid',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_password',
)

http_auth = (opensearch_user_id, opensearch_user_password) # Master username, Master password

### Index 이름 셋팅
- 이전 노트북 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 생성된 OpenSearch Index name 입력

In [17]:
#index_name = "v13-genai-poc-knox-titan-eval-sample-0.05-1024c-256o"
index_name = "v16-genai-poc-knox-eval-parent-doc-retriever"

### OpenSearch Client 생성

In [18]:
from utils.opensearch import opensearch_utils

In [19]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

### LangChain OpenSearch VectorStore 생성

In [20]:
from langchain.vectorstores import OpenSearchVectorSearch

In [21]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)
vector_db

## 5. Retriever 정의

In [22]:
from utils.rag import OpenSearchHybridSearchRetriever

In [23]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    os_client=os_client,
    index_name=index_name,
    llm_text=llm_text_light, # used in rag_fusion, hyde and reranker(num_tokens)
    llm_emb=llm_emb,

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for rank fusion
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.5, .5], # [for lexical, for semantic], Lexical, Semantic search 결과에 대한 최종 반영 비율 정의
    reranker=True, # enable reranker with reranker model
    reranker_endpoint_name=endpoint_name, # endpoint name for reranking model
    #rag_fusion=False, # enable rag_fusion
    #query_augmentation_size=3, # query_augmentation_size in rag_fusion

    # option for async search
    async_mode=True,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=False,
)

In [24]:
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents("knox")
search_hybrid_result

[Document(page_content='What is Knox Suite?. Knox Suite is a bundled offering designed to help enterprise IT admins better manage your fleet of devices. It includes individual services such as Knox Platform for Enterprise, Knox Mobile Enrollment, Knox Manage, Knox E-FOTA, and Knox Asset Intelligence. Key features of Knox Suite include: Secure - Ensure your business data is protected with managed security features at your control. Deploy - Enroll corporate devices to EMM automatically and securely, right out of the box. Manage - Enable comprehensive management over corporate devices and OS versions. Analyze - Strengthen in-depth device analytics for mobile productivity and asset management.', metadata={'source': 'all_processed_data.json', 'seq_num': 74, 'title': ' What is Knox Suite?', 'url': 'https://docs.samsungknox.com/admin/knox-suite/faq/#what-is-knox-suite', 'project': 'KS', 'last_updated': '2023-07-26', 'family_tree': 'parent', 'parent_id': None, 'id': '5c911c77-600b-404f-a343-8a

## 6. RAG chain 정의

In [25]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

### Prompting
- [TIP] Prompt의 instruction의 경우 한글보다 영어로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [26]:
prompt_template = """
\n\nHuman: Here is the context, inside <context></context> XML tags.

<context>
{context}
</context>

Only using the context as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.

Question:
{question}

If the answer is not in the context, just say "Could not find answer in given contexts."

\n\nAssistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

### Update Search Params (Optional)

In [33]:
opensearch_hybrid_retriever.update_search_params(
    k=5,
    minimum_should_match=0,
    filter=[],
    reranker=True,
    reranker_endpoint_name=endpoint_name,
    rag_fusion=False, # enable rag_fusion
    query_augmentation_size=3, # query_augmentation_size in rag_fusion
    hyde=False, # enable hyde
    hyde_query=["web_search"], # query type in hyde 
    parent_document=True,
    llm_text=llm_text, # used in rag_fusion, hyde and reranker
    verbose=False
)

In [34]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm_text,
    chain_type="stuff",
    retriever=opensearch_hybrid_retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": PROMPT,
        "verbose": False,
    },
    verbose=False
)

In [35]:
query = "How many characters are removed when the start position is 1 and the number of characters to remove is also 1?"
qa_chain(query)

{'query': 'How many characters are removed when the start position is 1 and the number of characters to remove is also 1?',
 'result': ' The number of characters removed when the start position is 1 and the number of characters to remove is 1 is 1 character. This is based on the "Remove characters" action in the context, which states: \n\n"When sending data, removes a specific number of characters from the given positions. You must provide the start position and the number of characters to remove."',
 'source_documents': [Document(page_content=". On the Knox Capture home screen, tap on the profile you want to create a rule for. 2. Under Keystroke output and data formatting, tap Advanced data formatting. 3. In the top right corner of the screen, tap +. 4. Enter a name for the rule, then tap Done. 5. Under Set criteria, configure the below settings as needed: Barcode type - The types of barcodes that the rule applies to. String length - The required length of scanned data to trigger the 

## 3.Evaluation
 - [langchain evaluator](https://python.langchain.com/docs/guides/evaluation/string/criteria_eval_chain)

### Load ground truth

In [36]:
import pandas as pd

In [37]:
ground_thruth = pd.read_csv("eval_dataset_v16.csv")

In [38]:
import time
import datetime
from langchain.evaluation import Criteria
from langchain.evaluation import EvaluatorType
from langchain.evaluation import load_evaluator

In [39]:
list(Criteria)

[<Criteria.CONCISENESS: 'conciseness'>,
 <Criteria.RELEVANCE: 'relevance'>,
 <Criteria.CORRECTNESS: 'correctness'>,
 <Criteria.COHERENCE: 'coherence'>,
 <Criteria.HARMFULNESS: 'harmfulness'>,
 <Criteria.MALICIOUSNESS: 'maliciousness'>,
 <Criteria.HELPFULNESS: 'helpfulness'>,
 <Criteria.CONTROVERSIALITY: 'controversiality'>,
 <Criteria.MISOGYNY: 'misogyny'>,
 <Criteria.CRIMINALITY: 'criminality'>,
 <Criteria.INSENSITIVITY: 'insensitivity'>,
 <Criteria.DEPTH: 'depth'>,
 <Criteria.CREATIVITY: 'creativity'>,
 <Criteria.DETAIL: 'detail'>]

In [40]:
evaluator = load_evaluator(
    EvaluatorType.LABELED_CRITERIA,
    llm=llm_text,
    criteria=Criteria.CORRECTNESS
)

### Check prompt for evaluation

In [41]:
print(evaluator.prompt.partial_variables)
print("==")
print(evaluator.prompt.template)

{'criteria': 'correctness: Is the submission correct, accurate, and factual?'}
==
You are assessing a submitted answer on a given task or input based on a set of criteria. Here is the data:
[BEGIN DATA]
***
[Input]: {input}
***
[Submission]: {output}
***
[Criteria]: {criteria}
***
[Reference]: {reference}
***
[END DATA]
Does the submission meet the Criteria? First, write out in a step by step manner your reasoning about each criterion to be sure that your conclusion is correct. Avoid simply stating the correct answers at the outset. Then print only the single character "Y" or "N" (without quotes or punctuation) on its own line corresponding to the correct answer of whether the submission meets all criteria. At the end, repeat just the letter again by itself on a new line.


In [42]:
ground_thruth.head()

,question,answer,doc_id,doc
0,What is the default mode of operation for Knox...,Admin mode is the default mode of operation fo...,7f17c8ef-7d22-44a7-b92e-b26f4716d3d0,. 3. Export profile settings as a configuratio...
1,What are the two types of tutorials mentioned ...,The tutorials mentioned in the provided contex...,6a70e00b-0e60-4dfa-a9cf-6d753d3eee7c,. Admin mode is the default mode of operation ...
2,- How can an IT admin configure camera scan tr...,An IT admin can set camera scan triggers in Kn...,709c077e-5440-4ef4-afc8-d96e4209f659,. Test mode - A feature that lets you test a s...
3,What step do you take in the Knox Capture prof...,To enable the device's flashlight to be used w...,2668595c-19a0-4e9a-9e38-25dbd455a85e,. Add or remove a barcode type of your choice....
4,- How would you summarize the key steps covere...,Knox Capture for IT admins is a workflow that ...,87a9dba8-d3a8-41c4-8fd9-eac0d1d50c61,. Save the updated settings and deploy. Summar...


In [ ]:
%%time
evaluation = []
parent_document = True
for idx, row in enumerate(ground_thruth.itertuples()):
    try:
        question, answer, doc, doc_id = getattr(row, "question"), getattr(row, "answer"), getattr(row, "doc"), getattr(row, "doc_id")

        start = time.time()
        response = qa_chain(question)
        elapsed = time.time() - start

        prediction = response["result"]
        retrieved_docs = {doc.page_content: idx+1 for idx, doc in enumerate(response["source_documents"])}
        contexts = "\n\n".join(retrieved_docs.keys())

        payload = json.dumps(
            {
                "inputs": [
                    {"text": answer, "text_pair": prediction},
                ]
            }
        )

        response = runtime_client.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType="application/json",
            Accept="application/json",
            Body=payload
        )
        out = json.loads(response['Body'].read().decode()) ## for json

        eval_result = evaluator.evaluate_strings(
            input=question,
            prediction=prediction,
            reference=answer,
        )

        reranker_sim = out[0]["score"]
        langchain_correctness = eval_result["score"]
        
        if not parent_document:
            if doc in retrieved_docs:
                true_context_rank = retrieved_docs[doc]
                has_right_context = 1
                mrr = 1/true_context_rank
            else:
                true_context_rank = has_right_context = 0
                mrr = 0
        else:
            for retrieved_doc in retrieved_docs:
                
                if doc in retrieved_doc:

                    true_context_rank = retrieved_docs[retrieved_doc]
                    has_right_context = 1
                    mrr = 1/true_context_rank
                    break
                else:
                    true_context_rank = has_right_context = 0
                    mrr = 0

        total_contexts = len(retrieved_docs)
        latency = datetime.timedelta(seconds=elapsed)

        print(idx)
        print(colored("Question: ", "red"), colored(question, "red"))
        print(colored("GroundTruth: ", "green"), colored(answer, "green"))
        print(colored("Prediction: ", "blue"), colored(prediction, "blue"))
        print(f'ReRanker similarity: {reranker_sim}')
        print(f'Langchain correctness: {langchain_correctness}')
        print(f'has_right_context: {has_right_context}')
        print(f'true_context_rank: {true_context_rank}/{total_contexts}')
        print(f'mrr: {mrr:.3f}')
        print(f'latency: {latency} secs')
        print(f'prediction length: {len(prediction)}')

        evaluation.append([question, answer, prediction, \
                           has_right_context, true_context_rank, total_contexts, mrr, \
                           reranker_sim, langchain_correctness, elapsed, contexts, len(prediction)])
    except:
        print ("err", idx)

eval_results = pd.DataFrame(
    evaluation,
    columns=["question", "true_answer", "prediction", \
             "has_right_context", "true_context_rank", "total_contexts", "mrr", \
             "reranker_sim", "langchain_correctness", "latency", "contexts", "pred_length"]
)
eval_results.to_csv("eval_results_parent_doc.csv", index=False)
#eval_results.to_pickle("eval_results.pkl")

0
Question:  What is the default mode of operation for Knox Capture?
GroundTruth:  Admin mode is the default mode of operation for Knox Capture. Therefore, a user cannot switch to Admin mode by activating a Knox Capture license, the app is exported in admin mode
Prediction:   Admin mode is the default mode of operation for Knox Capture.
ReRanker similarity: 0.9999275207519531
Langchain correctness: 1
has_right_context: 1
true_context_rank: 1/5
mrr: 1.000
latency: 0:00:05.501050 secs
prediction length: 62 secs
1
Question:  What are the two types of tutorials mentioned that provide guidance for new admins and end-users of Knox Capture?
GroundTruth:  The tutorials mentioned in the provided context are: 'Getting started tutorials' and 'While they don't cover every setting or scenario when using Knox Capture, they do provide a good starting point for learning the app's basics.
Prediction:   Admin get started and End-user get started.
ReRanker similarity: 0.04858165606856346
Langchain correc